## Instalar e importar módulos

In [1]:
#Beautiful Soup de bs4 y requests sirven para WebScraping (ya instalados con Anaconda)
#Selenium sirve para automatizar el navegador(desde Anaconda Prompt ejecutar: "pip install selenium")
#PyAutoGUI sirve para automatizar movimientos de mouse, clicks, teclado ("pip install pyautogui")
#pyperclip para trabajar con portapapeles ("pip install pyperclip")
#pandas para armar el dataframe, time para hacer pausas entre comandos

In [250]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
import pandas as pd
import pyautogui as pg
import pyperclip as pc
import time

In [3]:
#Parsers de Scraping: lxml y html5lib (ya instalados con Anaconda)
#Webdriver de Chrome: Ir a "https://sites.google.com/a/chromium.org/chromedriver/downloads" para descargar un WebDriver
#para usarlo con Selenium  
#Revisar que sea la misma versión, el mío es ChromeDriver 83.0.4103.39 para Windows, para revisar la versión correcta
#ir a las opciones a la esquina superior derecha de Chrome > Help > About Google Chrome

## Entrar a PowerCampus > Clases Virtuales con Selenium

In [280]:
usuarioUP="*"
contraseñaUP="*"
cursoUP="sem. m&"

browser = webdriver.Chrome("C:\Program Files (x86)\chromedriver83.exe") #Acá puse el driver
browser.get("https://autoservicio2.up.edu.pe/ss/Home.aspx")
barra_user = browser.find_element_by_id("ctl00_ucUserLogin_lvLoginUser_ucLoginUser_lcLoginUser_UserName")
barra_password = browser.find_element_by_id("ctl00_ucUserLogin_lvLoginUser_ucLoginUser_lcLoginUser_Password")
barra_user.send_keys(usuarioUP) #Acá poner su usuario UP
barra_password.send_keys(contraseñaUP) #Acá la contraseña
barra_password.send_keys(Keys.ENTER)
boton_clases = browser.find_element_by_link_text("Clases")
boton_clases.send_keys(Keys.ENTER)
boton_clasesvirtuales = browser.find_element_by_link_text("Clases Virtuales")
boton_clasesvirtuales.send_keys(Keys.ENTER)
time.sleep(0.2)
pg.hotkey('winleft','left')
time.sleep(0.2)
pg.press('esc')
pagina_base = browser.current_url

## Navegar cursos y obtener links con Beautiful Soup o requests

### Videos de un curso

In [281]:
boton_desplegable = browser.find_element_by_id("select2-searchCursos-container")
boton_desplegable.click()
busqueda = browser.find_element_by_class_name("select2-search__field")
busqueda_desplegable = browser.find_element_by_xpath('//*[@id="ctl00_Html1"]/body/span/span/span[1]/input')
busqueda_desplegable.send_keys(cursoUP) #Nombre del curso
busqueda_desplegable.send_keys(Keys.ENTER)

In [284]:
#Scraping
time.sleep(1.8)
pg.hotkey('ctrl','shift','i')
time.sleep(4)
for i in range(2):
    pg.press('up')
pg.hotkey('shift','f10')
time.sleep(1)
for i in range(5):
    pg.press('down')
pg.press('right')
for i in range(2):
    pg.press('down')
pg.press('enter')
pg.hotkey('ctrl','shift','i')
html = pc.paste()
soup = BeautifulSoup(html, "lxml") #soup es el objeto, lmxl es el parser
#Ahora lo guardo en un archivo 'html.html'
with open('html.html', "wb") as file:
    file.write(soup.prettify("utf-8"))

## Obtener nombres de cursos

In [285]:
cursoslista = soup.find('select', {"id":"searchCursos"})
cursostexto = cursoslista.contents
cursos=[] #Acá están los nombres de todos los cursos UP
for i in range(1,len(cursostexto)):
    cursos.append(cursostexto[i].text)

## Obtener cabeceras y datos

### Cabeceras

In [286]:
tabla = soup.find('table', {"class":"table","id":"tablaResultado"})
cabeceraslista = tabla.thead.tr
cabecerastexto = cabeceraslista.contents
cabeceras=[]
for i in range(1,len(cabecerastexto)-1,2):
    cabeceras.append(cabecerastexto[i].text)
cabeceras.append('Link2')

### Datos

In [287]:
datos = tabla.tbody
claseslista = datos.contents
curso=[]
seccion=[]
fecha=[]
nombreseccion=[]
duracion=[]
link=[] #Acá estarán los urls ca.bbcollab
link2=[] #Acá estarán los urls cloudfront
# claseslista[#].find_all('td') ubica clase número #
# El .text obtiene el contenido de la etiqueta; el a.get('href') obtiene el url del link

for i in range(len(claseslista)):
    curso.append((claseslista[i].find_all('td')[0]).text)
    seccion.append((claseslista[i].find_all('td')[1]).text)
    fecha.append((claseslista[i].find_all('td')[2]).text)
    nombreseccion.append((claseslista[i].find_all('td')[3]).text)
    duracion.append((claseslista[i].find_all('td')[4]).text)
    link.append((claseslista[i].find_all('td')[5]).a.get('href'))

#### Obtener los link2

In [288]:
#Esta subrutina es un while que espera hasta que aparezca una imagen en pantalla y mueve el cursor hacia ella
def esperaryencontrar(imagen):
    r=None
    while r is None:
        time.sleep(0.1)
        r=pg.locateCenterOnScreen(imagen)
        if r==None:
            print("Todavía no se encuentra su imagen %s" %imagen)
        else:
            pg.moveTo(r)

In [289]:
def continuarsiseencuentra(imagen):
    r4=None
    while r4 is None:
        time.sleep(1.5)
        r4=pg.locateCenterOnScreen(imagen)
        if r4==None:
            print("Todavía no se encuentra su imagen %s" %imagen)
        else:
            continue

In [290]:
#Esta subrutina es un while que espera hasta que aparezca una imagen en pantalla y hace click en ella
def esperaryclickear(imagen):
    r2=None
    while r2 is None:
        time.sleep(0.1)
        r2=pg.locateCenterOnScreen(imagen)
        if r2==None:
            print("Todavía no se encuentra su imagen %s" %imagen)
        else:
            pg.click(r2)

In [55]:
#Esta subrutina es un while que espera hasta que aparezca una imagen en pantalla y hace click derecho en ella
def esperaryclickearderecha(imagen):  
    r3=None
    while r3 is None:
        time.sleep(0.1)
        r3=pg.locateCenterOnScreen(imagen)
        if r3==None:
            print("Todavía no se encuentra su imagen %s" %imagen)
        else:
            pg.click(r3,button='right')

In [248]:
# browser = webdriver.Chrome("C:\Program Files (x86)\chromedriver83.exe")
# link2=[]
for j in range(len(link)):
    browser.get(link[j])
    if j==0:
        time.sleep(2)
        pg.hotkey('ctrl','shift','i')
        esperaryclickear('Network.png')
        time.sleep(1)
        for i in range(2):
            pg.press('tab')
        for i in range(5):
            pg.press('right')
        pg.press('enter')
        for i in range(5):
            time.sleep(0.2)
            pg.press('tab')
    continuarsiseencuentra('206.png')
    pg.press('down')
    time.sleep(1)
    pg.hotkey('shift','f10')
    time.sleep(1)
    pg.press('down')
    time.sleep(0.5)
    pg.press('enter')
    browser.switch_to.window(browser.window_handles[1])
    link2.append(browser.current_url)
    print("Se encontró el link2 %s/"%(j+1)+"%s "%(len(link))+("(%s%%)")%(round((j+1)*100/(len(link)),2)))
    pg.hotkey('ctrl','w')
    browser.switch_to.window(browser.window_handles[0])
time.sleep(1.5)
pg.hotkey('ctrl','shift','i')

Todavía no se encuentra su imagen Network.png
Se encontró el link2 1/11 (9.09%)
Todavía no se encuentra su imagen 206.png
Todavía no se encuentra su imagen 206.png
Se encontró el link2 2/11 (18.18%)
Todavía no se encuentra su imagen 206.png
Todavía no se encuentra su imagen 206.png
Se encontró el link2 3/11 (27.27%)
Todavía no se encuentra su imagen 206.png
Todavía no se encuentra su imagen 206.png
Todavía no se encuentra su imagen 206.png
Se encontró el link2 4/11 (36.36%)
Todavía no se encuentra su imagen 206.png
Todavía no se encuentra su imagen 206.png
Se encontró el link2 5/11 (45.45%)
Todavía no se encuentra su imagen 206.png
Todavía no se encuentra su imagen 206.png
Se encontró el link2 6/11 (54.55%)
Todavía no se encuentra su imagen 206.png
Todavía no se encuentra su imagen 206.png
Se encontró el link2 7/11 (63.64%)
Todavía no se encuentra su imagen 206.png
Todavía no se encuentra su imagen 206.png
Se encontró el link2 8/11 (72.73%)
Todavía no se encuentra su imagen 206.png
Tod

## Marco de datos con pandas y exportar como csv

In [249]:
datostot=[curso,seccion,fecha,nombreseccion,duracion,link,link2]
marco1 = pd.DataFrame()
for i in range(len(cabeceras)):
    marco1[cabeceras[i]]=datostot[i]
marco1

,Curso,Sección,Fecha,Nombre de la sesión,Duración,Link,Link2
0,Sem. M&A Case Studies,A,2020-03-31,Clase 1,1:44:14,https://ca.bbcollab.com/recording/5d4f1271523e...,https://dfz4hf1hz1uri.cloudfront.net/content/7...
1,Sem. M&A Case Studies,A,2020-04-07,Clase 2,1:53:37,https://ca.bbcollab.com/recording/93ba79d4636e...,https://dfz4hf1hz1uri.cloudfront.net/content/9...
2,Sem. M&A Case Studies,A,2020-04-14,Clase 3,1:51:07,https://ca.bbcollab.com/recording/1947a3f19e80...,https://dfz4hf1hz1uri.cloudfront.net/content/e...
3,Sem. M&A Case Studies,A,2020-04-21,Clase 4,1:44:29,https://ca.bbcollab.com/recording/bd9cc01425a1...,https://dfz4hf1hz1uri.cloudfront.net/content/2...
4,Sem. M&A Case Studies,A,2020-04-28,Clase 5,1:45:56,https://ca.bbcollab.com/recording/34b3a04641a8...,https://dfz4hf1hz1uri.cloudfront.net/content/9...
5,Sem. M&A Case Studies,A,2020-05-05,Clase 6,1:51:25,https://ca.bbcollab.com/recording/605da0b292b8...,https://dfz4hf1hz1uri.cloudfront.net/content/e...
6,Sem. M&A Case Studies,A,2020-05-12,Clase 7,1:45:20,https://ca.bbcollab.com/recording/5039a4cb7b08...,https://dfz4hf1hz1uri.cloudfront.net/content/d...
7,Sem. M&A Case Studies,A,2020-05-26,Clase 8,1:49:30,https://ca.bbcollab.com/recording/bba9429025c3...,https://dfz4hf1hz1uri.cloudfront.net/content/a...
8,Sem. M&A Case Studies,A,2020-06-02,Clase 9,1:50:38,https://ca.bbcollab.com/recording/72e06fd91030...,https://dfz4hf1hz1uri.cloudfront.net/content/4...
9,Sem. M&A Case Studies,A,2020-06-09,Clase 10,1:51:29,https://ca.bbcollab.com/recording/1ea632702386...,https://dfz4hf1hz1uri.cloudfront.net/content/7...


In [130]:
marco1.to_csv("marco1.csv", index=False, header=True)

In [261]:
y=marco1['Duración'].iloc[0]
y

'1:44:14'

In [275]:
int(y[0])*3600+int(y[2:3])*60+int(y[-2:-1])

3841

In [270]:
int(y[0])*3600

3600

In [276]:
print("este video dura %s" %marco1['Duración'].loc[1])

este video dura 1:53:37
